In [ ]:
def lowenstein_rule_bonds(atoms, indices):
    '''
    The Lowenstein rule states Al-O-Al bonds are forbidden in zeolites.
    '''
    
    # Get the indices of the Al atoms
    # alsel = AtomSelection.from_element(atoms, 'Al')
    # Alindices = alsel.indices

    for idx in indices:
        # the nn neighbours of the Al atom are:
        nn = np.where(bondmat[idx])[0]
        # the nn neighbours of the nn neighbours are:
        nn2 = np.where(bondmat[nn])[0]
        # if any of the nn2 neighbours is Al, return False
        if np.any(np.isin(nn2, indices)):
            return False
        
    return True


In [ ]:
def gen_added_structure(atoms_in, host = 'Al', defect = 'Na', rcut = 4.2, poisson_r=0.5, vdw_scale=0.8):
    '''
    Use the defectGen to generate a set of structures with an interstitial atom each.

    then filter these structures to only keep the N ones that maximise the distance between the interstitials 
    and minimise the distance between the interstitials and the defect atoms.
    '''

    # Generate the structures
    dG = defectGen(atoms_in, defect, poisson_r=poisson_r, vdw_scale=vdw_scale) # poisson_r controls the minimum distance between two defects, 
                                                       # vdw_scale the one between defects and existing atoms
                                                       # (proportional to both's Van der Waals radius)
    # combine all into one structure
    atoms = atoms_in.copy()
    for s in dG:
        atoms.append(s[0])

    hostsel = AtomSelection.from_element(atoms, host)
    defectsel = AtomSelection.from_element(atoms, defect)
    defectsel_indices = defectsel.indices

    print(f'Total number of Na atoms to try: {len(defectsel)}')

    # order the Al atoms by distance to the first Al atom
    dists = atoms.get_distances(hostsel.indices[0], hostsel.indices, mic=True)
    hostsel = hostsel[ np.argsort(dists) ]
    
    # choose the first defect as that closest to the first Al atom
    dists = atoms.get_distances(hostsel.indices[0], defectsel_indices, mic=True)
    chosen_defects = [defectsel_indices[np.argmin(dists)]]
    
    # loop over all other Al atoms
    for host_idx in hostsel.indices[1:]:
        # print(chosen_defects)
        # get the distances between the Al atom and the defects
        host_dists = atoms.get_distances(host_idx, defectsel_indices, mic=True)
        
        # which defect atoms lie within rcut of host?
        inds_within_rcut = defectsel_indices[host_dists < rcut]

        # remove these from the numpy array of defect indices
        # (so we can't choose them again)
        for i in inds_within_rcut:
            defectsel_indices = defectsel_indices[defectsel_indices != i]


        # of those, which is the furthest from the last chosen defect?
        new_ind = np.argmax(atoms.get_distances(chosen_defects[-1], list(inds_within_rcut), mic=True))

        # add the defect to the list of chosen defects
        chosen_defects.append(inds_within_rcut[new_ind])

    # Finally, return just the atoms that are in the list of chosen defects
    atoms_out = atoms_in.copy()
    for i in chosen_defects:
        atoms_out.append(atoms[i])


    # for each Al, find and print the nearest Na defect position and index
    defectsel = AtomSelection.from_element(atoms_out, defect)
    for i, host_idx in enumerate(hostsel.indices):
        dists = atoms_out.get_distances(host_idx, defectsel.indices, mic=True)
        print(f'{host} {i}: {dists.min():.3f} A from (nearest) {defect} {np.argmin(dists)}')

    
    return atoms_out



In [1]:
from ase.calculators.cp2k import CP2K
from ase.build import molecule


inp = """
&FORCE_EVAL
   METHOD Quickstep
   &DFT
      &QS
         METHOD XTB
            &XTB
            CHECK_ATOMIC_CHARGES F
            COULOMB_INTERACTION  T
            DO_EWALD  T
            HYDROGEN_STO_NG  4
            STO_NG  6
            TB3_INTERACTION T
            USE_HALOGEN_CORRECTION T
            DO_NONBONDED F
         &END XTB
      &END QS
   &END DFT
     &SUBSYS
      &KIND H
         BASIS_SET DZVP-MOLOPT-SR-GTH
         POTENTIAL GTH-LDA
      &END KIND
   &END SUBSYS
&END FORCE_EVAL
"""

# Create the calculator
calc = CP2K(basis_set=None,
                basis_set_file=None,
                max_scf=None,
                cutoff=None,
                force_eval_method=None,
                potential_file=None,
                poisson_solver=None,
                pseudo_potential=None,
                stress_tensor=False,
                xc=None,
                command = "docker run -u $(id -u ${USER}):$(id -g ${USER}) -v ${PWD}:/mnt --shm-size=1g -i cp2k/cp2k cp2k_shell",
                directory="temp",
                print_level = "MEDIUM",
                label='test_H2_inp', inp=inp)

h2 = molecule('H2', calculator=calc)
h2.center(vacuum=2.0)
energy = h2.get_potential_energy()

In [2]:
energy

-28.186310620517

In [ ]:
from ase.calculators.cp2k import CP2K


inp = """
&FORCE_EVAL
   METHOD Quickstep
   &DFT
      &QS
         METHOD DFTB
         &DFTB
         HB_SR_GAMMA 
         SELF_CONSISTENT    T
         DO_EWALD           T
         DISPERSION         T
         &PARAMETER
            SK_FILE Si O temp/matsci-0-3/Si-O.skf
            SK_FILE Si Si temp/matsci-0-3/Si-Si.skf
            SK_FILE O O temp/matsci-0-3/O-O.skf
            SK_FILE Si Al temp/matsci-0-3/Si-Al.skf
            SK_FILE Al Al temp/matsci-0-3/Al-Al.skf
            SK_FILE Al O temp/matsci-0-3/Al-O.skf
            SK_FILE Al Si temp/matsci-0-3/Al-Si.skf
            SK_FILE O Si temp/matsci-0-3/O-Si.skf
            SK_FILE O Al temp/matsci-0-3/O-Al.skf
            SK_FILE Na Na temp/matsci-0-3/Na-Na.skf
            SK_FILE Na O temp/matsci-0-3/Na-O.skf
            SK_FILE Na Al temp/matsci-0-3/Na-Al.skf
            SK_FILE Na Si temp/matsci-0-3/Na-Si.skf
            SK_FILE Si Na temp/matsci-0-3/Si-Na.skf
            SK_FILE Al Na temp/matsci-0-3/Al-Na.skf
            SK_FILE O Na temp/matsci-0-3/O-Na.skf
         &END PARAMETER
         &END DFTB
      &END QS
   &END DFT
&END FORCE_EVAL
"""

# Create the calculator
calc = CP2K(basis_set=None,
                basis_set_file=None,
                max_scf=None,
                cutoff=None,
                force_eval_method=None,
                potential_file=None,
                poisson_solver=None,
                pseudo_potential=None,
                stress_tensor=False,
                xc=None,
                command = "docker run -u $(id -u ${USER}):$(id -g ${USER}) -v ${PWD}:/mnt --shm-size=1g -i cp2k/cp2k cp2k_shell",
                directory="temp",
                print_level = "MEDIUM",
                label='test2_inp', inp=inp)

# atoms = zsm5.copy() # aColl.structures[0]
# atoms = gen_added_structure(aColl[order].structures[1])
# atoms.set_calculator(calc)
# energy = atoms.get_potential_energy()

In [ ]:
from ase.calculators.cp2k import CP2K


inp = """
&FORCE_EVAL
   METHOD Quickstep
   &DFT
      &QS
         METHOD XTB
            &XTB
            CHECK_ATOMIC_CHARGES F
            COULOMB_INTERACTION  T
            DO_EWALD  T
            HYDROGEN_STO_NG  4
            STO_NG  6
            TB3_INTERACTION T
            USE_HALOGEN_CORRECTION T
            DO_NONBONDED F
         &END XTB
      &END QS
   &END DFT
&END FORCE_EVAL
"""

# Create the calculator
calc = CP2K(basis_set=None,
                basis_set_file=None,
                max_scf=None,
                cutoff=None,
                force_eval_method=None,
                potential_file=None,
                poisson_solver=None,
                pseudo_potential=None,
                stress_tensor=False,
                xc=None,
                command = "docker run -u $(id -u ${USER}):$(id -g ${USER}) -v ${PWD}:/mnt --shm-size=1g -i cp2k/cp2k cp2k_shell",
                directory="temp",
                print_level = "MEDIUM",
                label='test3_inp', inp=inp)

# atoms = zsm5.copy() # aColl.structures[0]
# atoms = gen_added_structure(aColl[order].structures[0])
# atoms.set_calculator(calc)
# energy = atoms.get_potential_energy()